In [ ]:
from math import nan
from pathlib import Path

from gspread import service_account, Spreadsheet
from numpy import array
from pandas import DataFrame
from statsmodels.stats.inter_rater import fleiss_kappa

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1ud5ij645_khLMiydH3g_Dun9YCEeToWJnICw5a1RfNo/edit"
sheet_id = 0

In [ ]:
data_path = Path("../data")
figures_path = data_path / "figures"

In [ ]:
query = 19

In [ ]:
column_categories = {
    '2.1.A.': ["0", "1"],
    '2.1.B.': ["0", "1"],
    '2.2.A.': ["0", "1"],
    '2.2.B.': ["0", "1"],
    '2.3.': ["1", "2", "3", "4", "5", "6", "99"],
    '2.3.1.': ["1", "2", "3", "99"],
    '2.3.A.': ["0", "1"],
    '2.3.B.': ["0", "1"],
    '3.1.': ["1", "2", "3", "4", "99"],
    '3.2.': ["1", "2", "3", "4", "5", "99"],
    '3.3.': ["1", "2", "3", "4", "99"],
    '4.1.1.': ["0", "1"],
    '4.1.2.': ["0", "1"],
    '4.1.3.': ["0", "1"],
    '4.1.4.': ["0", "1"],
    '4.1.5.': ["0", "1"],
    '4.1.6.': ["0", "1"],
    '4.1.7.': ["0", "1"],
    '4.1.99.': ["0", "1"],
    '4.2.': ["1", "2", "3", "4", "99"],
    '5.1.': ["1", "2", "3", "4", "5", "6", "99"],
    '5.2.': ["1", "2", "3", "4", "99"],
    '5.3.': ["1", "2", "3", "4"],
    '5.4.': ["0", "1"],
    '6.1.': ["1", "2", "3", "99"],
    '6.2.': ["1", "2", "99"],
    '6.3.': ["1", "2", "99"],
    '7': ["1", "2", "3"],
    '7.1.': ["0", "1"],
}
columns = list(column_categories.keys())

In [ ]:
sheets = service_account("google-services.json")

In [ ]:
sheet: Spreadsheet = sheets.open_by_url(sheet_url)

In [ ]:
values = sheet.get_worksheet_by_id(sheet_id).get_all_values()

In [ ]:
def haltung_staerke(x: str) -> str:
    if x == "1" or x == "5":
        return "3"
    elif (x == "2" or 
          x == "4"):
        return "2"
    elif x == "3" or x == "6":
        return "1"
    elif x == "99":
        return "99"
    else:
        raise Exception(x)

In [ ]:
data = DataFrame(values[2:], columns=values[0])

# Spalten löschen
# del data["1.3."]
# del data["1.4."]
# del data["1.5."]
# del data["Relevanzkriterium"]
# del data["Haltung"]
# del data["Nutzbarkeit"]
# del data["Inhalt"]
# del data["Glaubwürdigkeit"]
# del data["Aktualität"]
# del data["QUALITÄT"]

data["1.6."] = data["1.6."].str.strip()
data["1.7."] = data["1.7."].str.strip()

# Umcodierung
data["2.1.A."] = ((data["2.1."] == "1") | (data["2.1."] == "3")).astype(int).astype(str)
data["2.1.B."] = ((data["2.1."] == "2") | (data["2.1."] == "3")).astype(int).astype(str)
# del data["2.1."]

# Umcodierung
data["2.2.A."] = ((data["2.2."] == "1") | (data["2.2."] == "3")).astype(int).astype(str)
data["2.2.B."] = ((data["2.2."] == "2") | (data["2.2."] == "3")).astype(int).astype(str)
# del data["2.2."]

data.loc[data["5.2."] == "99", "2.3."] = "99"
data["2.3.1."] = data["2.3."].map(haltung_staerke).astype(int).astype(str)
data["2.3.A."] = ((data["2.3."] == "1") | (data["2.3."] == "2")).astype(int).astype(str)
data["2.3.B."] = ((data["2.3."] == "4") | (data["2.3."] == "5")).astype(int).astype(str)

# Daten aufbereiten
data["4.1."] = data["4.1."].str.replace(";", ",")
data["4.1."] = data["4.1."].str.replace(" ", "")
data["4.1."] = data["4.1."].str.removesuffix(",")
data["4.1."] = data["4.1."].map(lambda text: set(text.split(",")))
for element in ["1", "2", "3", "4", "5", "6", "7", "99"]:
    data[f"4.1.{element}."] = data["4.1."].map(lambda labels: str(int(element in labels)))
# del data["4.1."]

# Umcodierung
data["5.4."] = data["5.4."].str.replace(";", ",")
data["5.4."] = data["5.4."].str.replace(" ", "")
data["5.4."] = data["5.4."].str.removesuffix(",")
data["5.4."] = data["5.4."].map(lambda text: set(text.split(",")))
data["5.4."] = data["5.4."].map(
    lambda labels: "1" in labels or "2" in labels or "3" in labels or "99" in labels).astype(int).astype(str)

# Umcodierung
data["7.1."] = ((data["7"] == "2") | (data["7"] == "3")).astype(int).astype(str)

data.head()

In [ ]:
agreement_data = data.copy()
agreement_data = agreement_data[agreement_data["1.1."] == str(query)]
agreement_data["Subject"] = agreement_data["1.1."].str.cat(agreement_data["1.2."], sep="-")
agreement_data = agreement_data[["Subject", "Codierer", *columns]]
agreement_data.sort_values(by=["Subject", "7"], inplace=True)
agreement_data.head()

In [ ]:
subjects = agreement_data["Subject"].unique()
subjects

In [ ]:
def column_fleiss_kappa(column: str) -> float:
    categories = column_categories[column]
    table = array([
        [
            len(agreement_data[(agreement_data["Subject"] == subject) & (agreement_data[column] == category)])
            for category in categories
        ]
        for subject in subjects
    ])
    return fleiss_kappa(table)

In [ ]:
agreement = DataFrame([
    {
        "Frage": column,
        "Fleiss' κ": column_fleiss_kappa(column),
    }
    for column in columns
])
agreement

In [ ]:
agreement.to_excel("agreement.xlsx")

In [ ]:
data.to_excel("umcodiert.xlsx")

In [ ]:
from pandas import Series
from collections import Counter

analytics_data = data.copy()


def majority_vote(items: Series):
    counter = Counter(items.tolist())
    _, most_common_count = counter.most_common(1)[0]
    most_common = {key for key in counter.keys() if counter[key] == most_common_count}
    most_common = list(sorted(most_common))
    return most_common[0] if len(most_common) >= 1 else nan


analytics_data = analytics_data.groupby(by=["1.1.", "1.2."]).aggregate(majority_vote).reset_index()
analytics_data

In [ ]:
analytics_data.to_excel("umcodiert_majority_vote.xlsx")

In [ ]:
def score_umfang(value: str) -> float:
    if value == "1":
        return 1
    elif value == "2":
        return 2
    elif value == "3":
        return 3
    elif value == "4":
        return 4
    elif value == "5":
        return 0
    else:
        return nan

In [ ]:
def score_sprache(value: str) -> float:
    if value == "1":
        return 2
    elif value == "2":
        return 1
    elif value == "3":
        return 0
    elif value == "4":
        return 0
    else:
        return nan

In [ ]:
def score_lesbarkeit(value: str) -> float:
    if value == "1":
        return 0
    elif value == "2":
        return 1
    elif value == "3":
        return 2
    elif value == "4":
        return 3
    else:
        return nan

In [ ]:
def score_quelle(value: str) -> float:
    if value == "1":
        return 2
    elif value == "2":
        return 2
    elif value == "3":
        return 0
    elif value == "4":
        return 1
    elif value == "5":
        return 1
    elif value == "6":
        return 0
    else:
        return nan

In [ ]:
def score_autor(value: str) -> float:
    if value == "1":
        return 2
    elif value == "2":
        return 1
    elif value == "3":
        return 0
    elif value == "4":
        return 1
    elif value == "99":
        return 1
    else:
        return nan

In [ ]:
def score_wahrheit(value: str) -> float:
    if value == "1":
        return 1
    elif value == "2":
        return 0
    elif value == "3":
        return 0.5
    elif value == "4":
        return 1
    else:
        return nan

In [ ]:
def score_datum(value: str) -> float:
    if value == "1":
        return 1
    elif value == "2":
        return 2
    elif value == "3":
        return 2
    elif value == "99":
        return 0
    else:
        return nan

In [ ]:
def score_updates(value: str) -> float:
    if value == "1":
        return 1
    elif value == "2":
        return 0
    elif value == "99":
        return 0
    else:
        return nan

In [ ]:
index_data = analytics_data.copy()
index_data["I-Haltung"] = (index_data["2.3.1."].astype(int).replace(99, nan) - 1) / 2
index_data["3.2.score"] = index_data["3.2."].map(score_umfang)
index_data["3.3.score"] = index_data["3.3."].map(score_sprache)
index_data["I-Inhalt"] = (index_data["3.3.score"] + index_data["3.2.score"]) / 6
index_data["4.2.score"] = index_data["4.2."].map(score_lesbarkeit)
index_data["I-Nutzerfreundlichkeit"] = (
                                               # index_data["4.1.3."].astype(int)
                                               # + index_data["4.1.4."].astype(int)
                                               # + index_data["4.1.6."].astype(int)
                                               # +
                                               index_data["4.2.score"]
                                       ) / 3#9
index_data["5.1.score"] = index_data["5.1."].map(score_quelle)
index_data["5.2.score"] = index_data["5.2."].map(score_autor)
index_data["5.3.score"] = index_data["5.3."].map(score_wahrheit)
index_data["I-Glaubwürdigkeit"] = ((
                                           (index_data["5.1.score"] * 2) +
                                           index_data["5.2.score"]
                                   ) * index_data["5.3.score"]) / 6
index_data["6.1.score"] = index_data["6.1."].map(score_datum)
index_data["6.2.score"] = index_data["6.2."].map(score_updates)
index_data["I-Aktualität"] = ((index_data["6.1.score"] * 2) + index_data["6.2.score"]) / 5
index_data["I-Qualität"] = (
                                   (index_data["I-Inhalt"] * 4) +
                                   (index_data["I-Nutzerfreundlichkeit"] * 4) +
                                   (index_data["I-Glaubwürdigkeit"] * 2) +
                                   index_data["I-Aktualität"]
                           ) / 11
index_data

In [ ]:
index_data.to_excel("scores.xlsx")

In [ ]:
from math import ceil
from matplotlib import pyplot as plt


def hist(col: str, bins: int):
    n, bins, patches = plt.hist(x=index_data[col], bins=bins, color='#0504aa', alpha=0.7, rwidth=0.85)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('Wert')
    plt.ylabel('Häufigkeit')
    maxfreq = n.max()
    # Set a clean upper y-axis limit.
    plt.xlim(xmin=0, xmax=1)
    plt.ylim(ymax=ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
    plt.savefig(figures_path / f"Histogramm_{col}.png")
    plt.savefig(figures_path / f"Histogramm_{col}.pdf")

In [ ]:
hist("I-Haltung", 3)

In [ ]:
hist("I-Inhalt", 6)

In [ ]:
hist("I-Nutzerfreundlichkeit", 4)

In [ ]:
hist("I-Glaubwürdigkeit", 3)

In [ ]:
hist("I-Aktualität", 3)

In [ ]:
hist("I-Qualität", 10)

In [ ]:
index_data["7.1."] = index_data["7.1."].astype(int)

In [ ]:
from seaborn import heatmap

corr_data = DataFrame()
corr_data[[
    "Haltung",
    "Inhalt",
    "Nutzerfreundlichkeit",
    "Glaubwürdigkeit",
    "Aktualität",
    "Qualität",
    "Relevanz"
]] = index_data[[
    "I-Haltung",
    "I-Inhalt",
    "I-Nutzerfreundlichkeit",
    "I-Glaubwürdigkeit",
    "I-Aktualität",
    "I-Qualität",
    "7.1."
]]
corr_data = corr_data.fillna(0)
corr = corr_data.corr()
corr

In [ ]:
heatmap(corr)
plt.savefig(figures_path / "Qualität_Korrelation.png")
plt.savefig(figures_path / "Qualität_Korrelation.pdf")

In [ ]:
index_data.groupby("Hintergrund").aggregate("mean")

In [ ]:
index_data.groupby("1.1.").aggregate("mean")  #[["I-Haltung", "I-Qualität"]]

In [ ]:
index_data.groupby("Codierer").aggregate("mean")

In [ ]:
heatmap(index_data.groupby("1.1.").aggregate("mean")[[
    "I-Haltung",
    "I-Inhalt",
    "I-Nutzerfreundlichkeit",
    "I-Glaubwürdigkeit",
    "I-Aktualität",
    "I-Qualität",
    "7.1."
]].corr())